In [2]:
from manim import *
from rigid_mechanics import *
config.media_width = "100%"

In [9]:
from typing import List
import pymunk

def curve_to_segments(
    func,
    space: pymunk.Space,
    thickness: float = 0.005,
    friction: float = 0,
    elasticity: float = 0,
    layer: int = 0xFFFFFFFF,
    collide_with: int = 0xFFFFFFFF
) -> List[pymunk.Segment]:
    points = func.points
    static_body = pymunk.Body(body_type=pymunk.Body.STATIC)
    segments = []

    for i in range(len(points) - 1):
        a = (points[i][0], points[i][1])
        b = (points[i + 1][0], points[i + 1][1])
        segment = pymunk.Segment(static_body, a, b, thickness)
        segment.friction = friction
        segment.elasticity = elasticity
        segment.filter = pymunk.ShapeFilter(categories=layer, mask=collide_with)
        segments.append(segment)

    space.add(static_body, *segments)
    return segments

def unit_normal(segment):
    if hasattr(segment, "get_start"):  # Manim Line
        a = segment.get_start()
        b = segment.get_end()
    elif hasattr(segment, "a"):  # pymunk.Segment
        a = np.array([*segment.a, 0.0])
        b = np.array([*segment.b, 0.0])
    else:
        raise TypeError("Segment must be a Manim Line or a pymunk.Segment")
    tangent = b - a
    tangent[:2] /= np.linalg.norm(tangent[:2])  # Normalize XY
    normal = np.array([-tangent[1], tangent[0], 0])  # 90° CCW
    return normal, a


In [57]:
%%manim -qh SlideScene
class SlideScene(SpaceScene, MovingCameraScene):
    def construct(self):
        start_point = Dot(-PI*RIGHT, z_index = 5)
        end_point = Dot(PI/2*RIGHT - PI/2*UP, z_index = 5)
        ball_start = start_point.get_center()+0.04*RIGHT
        #constants
        STRAIGHT_LAYER = 1 << 0
        ARC_1_LAYER = 1 << 1
        ARC_2_LAYER = 1 << 2
        BALL_RADIUS = 0.1
        BUFFER = 1.05*BALL_RADIUS
        
        start_marker = VGroup(
            start_point,
            Text("A", font_size = 25).next_to(start_point, UP)
        )
        end_marker = VGroup(
            end_point,
            Text("B", font_size = 25).next_to(end_point, UP)
        )
        
        #straight path
        straight_path = Line(
            start_point.get_center(), end_point.get_center(),
            color = BLUE,
        )
        straight_ball = Circle(
            radius = 0.1,
            color = straight_path.get_color()
        )
        first_seg = straight_path[0]  # First segment of straight path (Manim Line)
        normal, point = unit_normal(first_seg)
        straight_ball.move_to(point + BUFFER * normal)

        #arc 1 path
        arc_1 = ArcBetweenPoints (
                start_point.get_center(), end_point.get_center(),
                color = PURPLE,
                radius = 8,
                num_components = 10
            )
        arc_1_path = curve_to_segments(
            arc_1,
            self.space.space,
            layer = ARC_1_LAYER,
            collide_with = ARC_1_LAYER
        )
        arc_1_ball = Circle(
            radius = 0.1,
            color = arc_1.get_color()
        )
        first_seg = arc_1[0]  
        normal, point = unit_normal(first_seg)
        arc_1_ball.move_to(point + BUFFER * normal)

        #arc_2 path
        arc_2 = ArcBetweenPoints (
                start_point.get_center(), end_point.get_center(),
                color = YELLOW,
                radius = 4,
                num_components = 10
            )
        arc_2_path = curve_to_segments(
            arc_2, 
            self.space.space,
            layer = ARC_2_LAYER,
            collide_with = ARC_2_LAYER
        )
        arc_2_ball = Circle(
            radius = 0.1,
            color = arc_2.get_color()
        )
        first_seg = arc_2_path[0]  
        normal, point = unit_normal(first_seg)
        arc_2_ball.move_to(point + BUFFER * normal)

        #adjust cam
        self.camera.frame.move_to(straight_path.get_midpoint())   
        self.wait(5)
        self.play(FadeIn(start_marker), FadeIn(end_marker))
        self.wait(5)
        self.play(Create(straight_path))
        self.wait(0.5)
        self.play(Create(arc_1))
        self.wait(0.5)
        self.play(Create(arc_2))

        self.play(
            Create(straight_ball), 
            Create(arc_1_ball),
            Create(arc_2_ball),
        )
        self.wait(0.3)
        
        self.make_rigid_body(straight_ball, layer = STRAIGHT_LAYER, collide_with = STRAIGHT_LAYER)
        self.make_static_body(straight_path, layer = STRAIGHT_LAYER, collide_with = STRAIGHT_LAYER)

        self.make_rigid_body(arc_1_ball, layer = ARC_1_LAYER, collide_with = ARC_1_LAYER)

        self.make_rigid_body(arc_2_ball, layer = ARC_2_LAYER, collide_with = ARC_2_LAYER)
        self.wait(5)
    

Manim Community v0.19.0

[06/09/25 22:37:21] INFO     Animation 0 : Partial movie file written in                   ]8;id=420574;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=371995;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\partial_movie_files\SlideS                         
                             cene\2307254328_3543312089_2751100354.mp4'                                            

[06/09/25 22:37:23] INFO     Animation 1 : Partial movie file written in                   ]8;id=283309;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=607631;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\partial_movie_files\SlideS                         
                             cene\2307254328_496106045_2170640778.mp4'                                             

[06/09/25 22:37:31] INFO     Animation 2 : Partial movie file written in                   ]8;id=941062;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=718674;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\partial_movie_files\SlideS                         
                             cene\110484278_3168159723_9549443.mp4'                                                

[06/09/25 22:37:32] INFO     Animation 3 : Partial movie file written in                   ]8;id=445128;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=147970;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\partial_movie_files\SlideS                         
                             cene\110484278_3292034566_641397126.mp4'                                              

...

[06/09/25 22:37:40] INFO     Animation 9 : Partial movie file written in                   ]8;id=765942;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=578438;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\partial_movie_files\SlideS                         
                             cene\110484278_2136834830_3739455458.mp4'                                             

[06/09/25 22:37:48] INFO     Animation 10 : Partial movie file written in                  ]8;id=389106;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=965;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\partial_movie_files\SlideS                         
                             cene\110484278_3543312089_2692560071.mp4'                                             

                    INFO     Combining to Movie file.                                      ]8;id=949405;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=723292;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#739\739]8;;\

[06/09/25 22:37:49] INFO                                                                   ]8;id=953850;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=444925;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\SlideScene.mp4'                                    
                                                                                                                   

                    INFO     Rendered SlideScene                                                       ]8;id=405363;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene.py\scene.py]8;;\:]8;id=126971;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene.py#255\255]8;;\
                             Played 11 animations                                                                  

In [23]:
%%manim -qh CompScene

class CompScene(MovingCameraScene):
    def construct(self):
        start_point = Dot(-PI*RIGHT, z_index = 5)
        end_point = Dot(PI/2*RIGHT - PI/2*UP, z_index = 5)
        
        start_marker = VGroup(
            start_point,
            Text("A", font_size = 25).next_to(start_point, UP)
        )
        end_marker = VGroup(
            end_point,
            Text("B", font_size = 25).next_to(end_point, UP)
        )
        #adjust cam
        self.camera.frame.move_to(midpoint(start_point.get_center(), end_point.get_center()))

        old_straight = Line(
            start_point.get_center(), end_point.get_center(),
            color = BLUE,
        )
        old_arc = ArcBetweenPoints (
                start_point.get_center(), end_point.get_center(),
                color = PURPLE,
                radius = 8,
                num_components = 10
            )
        arc_1 = ArcBetweenPoints (
                start_point.get_center(), end_point.get_center(),
                color = YELLOW,
                radius = 4,
                num_components = 10
            )
        arc_2 = ArcBetweenPoints (
                start_point.get_center(), end_point.get_center(),
                color = YELLOW,
                radius = 8,
                num_components = 10
            )
        arc_3 = ArcBetweenPoints (
                start_point.get_center(), end_point.get_center(),
                color = YELLOW,
                radius = 3.5,
                num_components = 10
            )
        arc_4 = ParametricFunction (
            lambda t: (t*t - PI, -PI*t/(2*np.sqrt(1.5*PI)), 0),
            color = YELLOW,
            t_range = (0, np.sqrt(1.5*PI), 0.01)
        )
        underline = Line(-PI*RIGHT - PI/2*UP, PI/2*RIGHT - PI/2*UP, color = YELLOW)
        title_card = Text("Brachistochrone")
        title_card.next_to(underline.get_midpoint(), UP, aligned_edge=DOWN)
        
        self.add (
            start_marker,
            end_marker, 
            arc_1,
            old_straight, 
            old_arc
        )
        self.play(FadeOut(old_straight, old_arc))
        self.wait(2)
        self.play(Transform(arc_1, arc_2))
        self.wait(0.5)
        self.play(Transform(arc_1, arc_3))
        self.wait(0.5)
        self.play(Transform(arc_1, arc_4))
        self.wait(1)
        self.play(FadeOut(start_marker, end_marker))
        self.wait(0.1)
        self.play(Transform(arc_1, underline), self.camera.frame.animate.move_to(underline.get_midpoint()))
        self.wait(0.5)
        self.play(Write(title_card))
        self.wait(1)
        self.play(FadeOut(title_card))
        self.wait()

Manim Community v0.19.0

[06/28/25 18:47:13] INFO     Animation 0 : Using cached data (hash :                           ]8;id=191755;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=935518;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             2307254328_3320376727_3016365112)                                                     

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=118856;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=78491;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             110484278_1831374817_2219567007)                                                      

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=146910;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=822662;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             110484278_4202749462_803509600)                                                       

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=668448;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=741560;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             110484278_422628456_1124949263)                                                       

...

[06/28/25 18:47:23] INFO     Animation 16 : Partial movie file written in                  ]8;id=332309;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=415410;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\partial_movie_files\CompSc                         
                             ene\2863832697_1441776934_404280786.mp4'                                              

[06/28/25 18:47:25] INFO     Animation 17 : Partial movie file written in                  ]8;id=607636;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=355969;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\partial_movie_files\CompSc                         
                             ene\2863832697_2968904986_2571251867.mp4'                                             

                    INFO     Combining to Movie file.                                      ]8;id=696139;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=249367;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=215710;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=204063;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\CompScene.mp4'                                     
                                                                                                                   

                    INFO     Rendered CompScene                                                        ]8;id=720927;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene.py\scene.py]8;;\:]8;id=517360;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene.py#255\255]8;;\
                             Played 18 animations                                                                  

In [16]:
%%manim -qh ProblemScene

class ProblemScene(MovingCameraScene):
    def construct(self):
        def normalize(vec):
            return vec / np.linalg.norm(vec)
            
        underline = Line(-PI*RIGHT-PI/2*UP, PI/2*RIGHT-PI/2*UP, color = YELLOW)
        
        road = Rectangle(
            width = config.frame_width, 
            height = 4,
            fill_color = YELLOW,
            fill_opacity = 1,
            stroke_color = YELLOW
        ).next_to(underline, DOWN, aligned_edge = UP, buff = 0)
        
        path = underline.copy().set(width = config.frame_width).rotate(-20*DEGREES)
        normal = normalize(np.cos(70*DEGREES)*RIGHT + np.sin(70*DEGREES)*UP)
        tangent = rotate_vector(normal, -90*DEGREES)

        ball = Circle(
            radius = 0.2,
            stroke_color = BLUE,
            fill_opacity = 0.5,
            fill_color = BLUE
        ).next_to(underline.get_midpoint() + 0.29*normal, buff=0)
        wheel = DashedVMobject(ball)
        wheel.set_fill(BLUE, 0.5)

        velocity = Arrow(ball.get_center(), ball.get_center() + 0.5*tangent, buff = 0)
        gravity = Arrow(ball.get_center(), ball.get_center() + 0.5*DOWN, buff = 0)
        normal_force = Arrow(ball.get_center(), ball.get_center() + 0.5*normal, buff = 0)
        self.add(underline)
        self.camera.frame.move_to(path.get_midpoint())
        self.play(self.camera.frame.animate.set(width = underline.width), Create(road))
        self.remove(underline)
        self.play(
            Rotating(road, radians = -20*DEGREES),
            DrawBorderThenFill(ball),
            run_time = 0.5,
            rate_func = smooth
        )
        self.wait(1)
        self.play(
            GrowArrow(velocity),
            Write(Tex(r"$v(t)$",font_size=20, color = RED).next_to(velocity, tangent, buff=0.1))
        )
        self.wait(0.5)
        self.play(
            GrowArrow(gravity),
            Write(Tex(r"$mg$", font_size=20, color = PURPLE).next_to(gravity, DOWN, buff=0.1))
        )
        self.wait(0.5)
        self.play(
            GrowArrow(normal_force),
            Write(Tex(r"$N(s)$", font_size=20, color = GREEN).next_to(normal_force, normal, buff=0.1))
        )
        self.wait(0.5)
        self.add(ball.copy().set_style(stroke_width = 0))
        self.play(
            Transform(ball, wheel)
        )
        self.play(
            Rotating(ball, radians = -2*PI),
            Write(Tex(r"$\omega(t)$", font_size=20, color = LIGHT_PINK).next_to(ball.get_center(), tangent + normal , buff=0.2))
        )
        self.play(
            *[FadeOut(mob)for mob in self.mobjects]
        )
        self.wait(5)
        

Manim Community v0.19.0

[06/28/25 18:43:24] INFO     Animation 0 : Using cached data (hash :                           ]8;id=364475;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=366004;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             4200382437_539736939_1991566735)                                                      

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=885602;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=727886;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             2912025512_442388385_3608379826)                                                      

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=299515;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=606513;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             2912025512_548731573_2201191785)                                                      

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=649306;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=738283;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             2912025512_1983459758_1127874925)                                                     

...

                    INFO     Animation 11 : Using cached data (hash :                          ]8;id=331311;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=39855;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             2912025512_616436953_3262047714)                                                      

                    INFO     Animation 12 : Using cached data (hash :                          ]8;id=75668;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=794768;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3080718480_677551340_631829896)                                                       

                    INFO     Combining to Movie file.                                      ]8;id=94355;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=816451;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#739\739]8;;\

[06/28/25 18:43:26] INFO                                                                   ]8;id=585107;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=37210;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\ProblemScene.mp4'                                  
                                                                                                                   

                    INFO     Rendered ProblemScene                                                     ]8;id=890151;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene.py\scene.py]8;;\:]8;id=456957;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene.py#255\255]8;;\
                             Played 13 animations                                                                  

In [24]:
%%manim -qh LightScene
class LightScene(Scene):
    def construct(self):
        question = Text("Where have we seen this before?")
        answer = Text("Light!", color = YELLOW)
        self.play(Write(question))
        self.play(
                ShowPassingFlash(
                    SurroundingRectangle(question, color = YELLOW, corner_radius=0.2),
                ),
                run_time=2
            )
        self.wait(6)
        self.play(FadeOut(question), FadeIn(answer))
        self.play(Flash(UP))
        self.wait(2)
        self.play(FadeOut(answer))
        self.wait(1)
        

Manim Community v0.19.0

[06/09/25 21:15:05] INFO     Animation 0 : Using cached data (hash :                           ]8;id=305612;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=279299;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3529212410_1121499409_223132457)                                                      

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=204917;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=555877;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3040924799_3310699706_2776372942)                                                     

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=311878;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=365087;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3040924799_2584036258_1184133186)                                                     

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=378344;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=149366;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3040924799_3262929134_3781214501)                                                     

...

[06/09/25 21:15:10] INFO     Animation 6 : Partial movie file written in                   ]8;id=701342;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=126743;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\partial_movie_files\LightS                         
                             cene\3040924799_43107254_1691007062.mp4'                                              

[06/09/25 21:15:11] INFO     Animation 7 : Partial movie file written in                   ]8;id=388863;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=270199;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\partial_movie_files\LightS                         
                             cene\3040924799_1839128356_3278671325.mp4'                                            

                    INFO     Combining to Movie file.                                      ]8;id=666636;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=142087;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=407189;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=427550;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\LightScene.mp4'                                    
                                                                                                                   

                    INFO     Rendered LightScene                                                       ]8;id=482629;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene.py\scene.py]8;;\:]8;id=820723;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene.py#255\255]8;;\
                             Played 8 animations                                                                   

In [36]:
%%manim -qh Refraction
class Refraction(Scene):
    def construct(self):
        cup_left = Line(2*UL, 2*DL)
        cup_bottom = Line(2*DL, 2*DR)
        cup_right = Line(2*DR, 2*UR)
        cup = VGroup(cup_left, cup_bottom, cup_right)
        start = Dot(-1.5*RIGHT + 1.5*UP)
        start_label = Text("A", font_size = 25).next_to(start, UP)
        end = Dot(1.5*RIGHT - 1.5*UP)
        end_label = Text("B", font_size = 25).next_to(end, UP)
    
        boundary = Line(cup_left.get_midpoint(), cup_right.get_midpoint())
        
        water = Rectangle(
            fill_opacity = 0.4,
            fill_color = BLUE, 
            width = 4, 
            height = 2,
            stroke_color = BLUE,
            z_index = -1
        )
        water.next_to(cup_bottom, UP, buff = 0, aligned_edge = DOWN)
        refraction_point = boundary.point_from_proportion(0.65)
        incident_ray = Arrow(
            start.get_center(),
            refraction_point,
            color = YELLOW,
            tip_length = 0.25,
            stroke_width = 2,
            buff = 0
        )
        incident_ray.put_start_and_end_on(
            incident_ray.get_start(),
            incident_ray.get_end() + incident_ray.get_default_tip_length()*incident_ray.get_unit_vector()
        )
        incident_ray.tip.shift(-0.5*incident_ray.get_length()*incident_ray.get_unit_vector())

        refracted_ray = Arrow(
            refraction_point,
            end.get_center(),
            color = YELLOW,
            stroke_width = 2,
            tip_length = 0.25,
            buff = 0
        )
        refracted_ray.put_start_and_end_on(
            refracted_ray.get_start(),
            refracted_ray.get_end() + refracted_ray.get_default_tip_length()*refracted_ray.get_unit_vector()
        )
        refracted_ray.tip.shift(-0.5*refracted_ray.get_length()*refracted_ray.get_unit_vector())
        
        water_ray = Arrow (
            start.get_center() + 3*LEFT,
            end.get_center() + 3*LEFT,
            color = YELLOW,
            stroke_width = 2,
            tip_length = 0.25,
            buff = 0
        )

        normal = DashedLine(refraction_point + 0.75*UP, refraction_point - 0.75*UP, dash_length = 0.1)
        angle_of_incidence = Angle(incident_ray, normal, quadrant = (-1,-1), other_angle = True)
        angle_of_refraction = Angle(normal, refracted_ray)

        v_1_label = Tex(r"$v_1$", font_size = 30).next_to(incident_ray.tip, RIGHT)
        v_2_label = Tex(r"$v_2$", font_size = 30).next_to(refracted_ray.tip, UR, buff = 0)
        alpha_1_label = Tex(r"$\alpha_1$", font_size = 30).next_to(refraction_point, UR, buff = 0.1)
        alpha_2_label = Tex(r"$\alpha_2$", font_size = 30).next_to(refraction_point, DL, buff = 0.1)

        snells_law = MathTex("\\frac{\\sin(\\alpha_1)}{v_1}", "=", "\\frac{\\sin(\\alpha_2)}{v_2}")
        snells_law.move_to(2*RIGHT)
        snells_law_title = Text("(Snell's Law of Refraction)", font_size = 25, color = RED)
        snells_law_title.next_to(snells_law, DOWN, aligned_edge = LEFT)
        
        self.play(Create(cup), DrawBorderThenFill(water))
        self.wait(1)
        self.play(FadeIn(start, start_label, end, end_label))
        self.wait(5)
        self.play(
            GrowArrow(incident_ray), 
            Write(v_1_label)
        )
        self.wait(2)
        self.play(
            GrowArrow(refracted_ray),
            Write(v_2_label)
        )
        self.wait(2)
        self.play(Create(normal))
        self.wait(0.5)
        self.play(
            Create(angle_of_incidence), 
            Create(angle_of_refraction)
        )
        self.play(
            Write(alpha_1_label),
            Write(alpha_2_label)
        )
        self.wait(2)
        self.play(*[mob.animate.shift(3*LEFT) for mob in self.mobjects])
        self.wait(1)
        self.play(Write(snells_law,  run_time = 3))
        self.play(FocusOn(snells_law))
        self.wait(0.5)
        self.play(FadeIn(snells_law_title))
        self.wait(6)
        self.play(
            FadeOut(
                snells_law_title,
                snells_law,
                alpha_1_label,
                alpha_2_label,
                angle_of_refraction,
                angle_of_incidence,
                normal,
                incident_ray,
                refracted_ray,
                v_1_label,
                v_2_label
            )
        )
        self.wait(4)
        self.play(
            water.animate.shift(DOWN).stretch_to_fit_height(0)
        )
        self.wait(1)
        self.play(GrowArrow(water_ray))
        self.wait(1)
        self.play(FadeOut(water_ray))
        self.wait(4)
        

Manim Community v0.19.0

[06/09/25 21:29:43] INFO     Animation 0 : Using cached data (hash :                           ]8;id=365247;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=354557;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3529212410_477372023_223132457)                                                       

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=434872;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=684647;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3040924799_1839128356_4265257489)                                                     

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=13158;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=116329;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3040924799_200712905_388071822)                                                       

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=515441;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=721910;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3040924799_3644383913_940435913)                                                      

...

[06/09/25 21:29:55] INFO     Animation 26 : Partial movie file written in                  ]8;id=961099;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=666974;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\partial_movie_files\Refrac                         
                             tion\3040924799_817823997_3784008173.mp4'                                             

[06/09/25 21:30:04] INFO     Animation 27 : Partial movie file written in                  ]8;id=339041;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=683766;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#588\588]8;;\
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\partial_movie_files\Refrac                         
                             tion\3040924799_2364698678_747679130.mp4'                                             

                    INFO     Combining to Movie file.                                      ]8;id=863059;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=639061;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#739\739]8;;\

[06/09/25 21:30:05] INFO                                                                   ]8;id=131655;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=256930;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\Refraction.mp4'                                    
                                                                                                                   

                    INFO     Rendered Refraction                                                       ]8;id=712209;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene.py\scene.py]8;;\:]8;id=274959;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene.py#255\255]8;;\
                             Played 28 animations                                                                  

In [41]:
%%manim -qh VaryMedium

from scipy.interpolate import make_interp_spline

def get_top_edge(rect):
    return Line(rect.get_corner(UL), rect.get_corner(UR))
    
def easing_proportions(start, end, num):
    linear = np.linspace(0, 1, num)
    eased = np.sqrt(linear)  # square root = fast rise, slow tail
    return start + (end - start) * eased
    
def first_last_boundaries(rects: VGroup, A: Dot, B: Dot):
    A_y = A.get_y()
    B_y = B.get_y()

    tops_y = [rect.get_top()[1] for rect in rects]
    bottoms_y = [rect.get_bottom()[1] for rect in rects]

    highest_below_A = max([i for i, top in enumerate(tops_y) if top < A_y])
    lowest_above_B = min(i for i, top in enumerate(tops_y) if top > B_y) 

    return highest_below_A, lowest_above_B

class VaryMedium(Scene):
    def construct(self):     
        cup_left = Line(2*UL, 2*DL)
        cup_bottom = Line(2*DL, 2*DR)
        cup_right = Line(2*DR, 2*UR)
        cup = VGroup(cup_left, cup_bottom, cup_right).shift(3*LEFT).set_z_index(5)
        start = Dot(-1.5*RIGHT + 1.5*UP).shift(3*LEFT)
        start_label = Text("A", font_size = 25).next_to(start, UP)
        end = Dot(1.5*RIGHT - 1.5*UP).shift(3*LEFT)
        end_label = Text("B", font_size = 25).next_to(end, UP)
        square_side = 4

        water = Square(
            side_length = square_side,
            fill_opacity = 0.4,
            fill_color = BLUE, 
            stroke_color = BLUE, 
            z_index = -1
        ).next_to(cup_bottom, UP, buff = 0, aligned_edge = DOWN)
        
        # Create horizontal subdivisions
        dy_values = [1 / i for i in range(1, 6)]  # list of different dy values
        bottom = water.get_bottom()
        left = water.get_left()[0]
        rect_groups = VGroup()

        for dy in dy_values:
            rects = VGroup()
            y_vals = np.arange(0, square_side, dy)
            indices = np.linspace( 1, 2, len(y_vals) )
            
            for y in y_vals:
                rect = Rectangle(
                    width=square_side,
                    height=dy,
                    stroke_color=BLACK if dy > 0.2 else None,
                    stroke_width=0.5 if dy > 0.2 else 0,
                    fill_opacity=0.4,
                ).set_fill(interpolate_color(ORANGE, BLUE, y / square_side))
                # Place rectangle at correct vertical position
                rect.move_to(bottom + UP * (y + dy / 2))
                rects.add(rect)
                
            rect_groups.add(rects)

            boundaries = [
                first_last_boundaries(rects, start, end)
                for rects in rect_groups
            ]

            
            refraction_points = [
                easing_proportions(start=0.5, end=0.86, num = boundaries[i][0] + 1 - boundaries[i][1])
                for i in range(len(rect_groups))
            ]
            
            paths = VGroup()
            for i in range(len(rect_groups)):
                proportions = refraction_points[i]
                rects = rect_groups[i]
                
                hi, lo = boundaries[i]
                rect_indices = range(hi, lo - 1, -1)
            
                # Match proportions with rects top-to-bottom
                path = Line(color=YELLOW, stroke_width=2).set_points_as_corners([
                    start.get_center(),
                    *[
                        get_top_edge(rects[r]).point_from_proportion(p)
                        for r, p in zip(rect_indices, proportions)
                    ],
                    end.get_center()
                ])
                paths.add(path)

        tangent_point = paths[len(paths)-1].get_anchors()[3]
        tangent = Line(tangent_point + 0.5*UL, tangent_point + 0.5*DR, stroke_width = 1)
        normal = DashedLine(tangent_point + 0.5* UP, tangent_point + 0.5*DOWN, stroke_opacity = 0.5)
        angle = Angle(tangent, normal, quadrant = (-1,-1), other_angle = True, color = RED, stroke_width = 2)
        angle_label = Tex(r"$\alpha$", font_size = 25).next_to(angle, RIGHT, buff = 0.1)
        velocity = Arrow(tangent_point, tangent.get_end(), buff = 0, tip_length = 0.5)
        velocity_label = Tex(r"$v$", font_size = 25).next_to(tangent.get_end(), RIGHT, buff = 0.1)

        snells_law = MathTex("\\frac{\\sin(\\alpha)}{v}", "=", "\\text{constant}")
        snells_law.move_to(2*RIGHT)
        prop = Text("(Property of the Brachistochrone)", font_size = 25, color = RED)
        prop.next_to(snells_law, DOWN, aligned_edge = LEFT)
        
        self.add(cup, start, start_label, end, end_label)

        r = rect_groups[0]
        p = paths[0]
        self.play(FadeIn(r))
        self.play(Create(p))


        for i in range(len(rect_groups)):
            self.play(
                Transform(r, rect_groups[i]),
                Transform(p, paths[i]),
                run_time = 0.5
            )
            self.wait(1)
        self.wait(2)
        self.play(
            Create(tangent),
            Create(normal)
        )
        self.wait(0.5)
        self.play(
            Create(angle),
            Write(angle_label)
        )
        self.play(
            GrowArrow(velocity),
            Write(velocity_label)
        )
        self.wait(2)
        self.play(Write(snells_law))
        self.wait(3)
        self.play(FadeIn(prop))
        self.wait(3)
        self.play(FadeOut(*self.mobjects))
        self.wait(2)

Manim Community v0.19.0

[06/09/25 21:34:36] INFO     Animation 0 : Using cached data (hash :                           ]8;id=306541;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=417542;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3529212410_4121500563_3758431623)                                                     

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=39348;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=40629;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3040924799_7719106_825568934)                                                         

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=405603;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=937699;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3040924799_1972142234_517702335)                                                      

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=901013;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=13897;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3040924799_1839128356_127499508)                                                      

...

                    INFO     Animation 23 : Using cached data (hash :                          ]8;id=295425;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=984977;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3040924799_2408024828_631829896)                                                      

                    INFO     Combining to Movie file.                                      ]8;id=444185;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=30883;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=798506;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=225994;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\VaryMedium.mp4'                                    
                                                                                                                   

                    INFO     The partial movie directory is full (> 100 files). Therefore, ]8;id=799181;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=46830;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#856\856]8;;\
                             manim has removed the 1 oldest file(s). You can change this                           
                             behaviour by changing max_files_cached in config.                                     

                    INFO     Rendered VaryMedium                                                       ]8;id=112242;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene.py\scene.py]8;;\:]8;id=63236;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene.py#255\255]8;;\
                             Played 24 animations                                                                  

In [45]:
%%manim -qh ProblemSolving

class ProblemSolving(MovingCameraScene):
    def construct(self):
        def cycloid_func(t):
            return np.array([
                np.sin(t) - t + axes.get_origin()[0] + TAU,   # x
                np.cos(t) - 1 + axes.get_origin()[1], # y
                0
            ])
            
        # Define axes
        axes = Axes(
            x_range=[0, 5, 1],
            y_range=[0, 3, 1],
            x_length=6,
            y_length=4
        ).shift(UL).rotate(angle=PI,axis=RIGHT)
    
        cycloid = ParametricFunction(
            cycloid_func,
            t_range=(2.5,2*PI,0.1),
            # Domain <---| |----> resolution
            color=YELLOW,
            z_index = -1
        )
        t_ball = (2.5+2*PI)/2 #t value
        
        start = cycloid.get_end()
        end = cycloid.get_start()
        start_point = VGroup(Dot(start), Text("A", font_size = 30).next_to(start, UP, buff=0.2))
        end_point = VGroup(Dot(end), Text("B", font_size = 30).next_to(end, UP, buff=0.2))
        ball = Dot(cycloid_func(t_ball), color = RED)
        y_line = DashedLine(
            ball.get_x()*RIGHT + start*UP,
            ball.get_x()*RIGHT + ball.get_y()*UP,
            stroke_width = 2,
            buff = 0.1
        )
        y_line_invis = Line(
            ball.get_x()*RIGHT + start*UP,
            ball.get_x()*RIGHT + ball.get_y()*UP,
        )
        y_line_extension = DashedLine(
            ball.get_x()*RIGHT + ball.get_y()*UP,
            ball.get_x()*RIGHT + (ball.get_y()-1)*UP,
            stroke_width = 2,
            buff = 0.1
        )
        y_label = Tex(r"$y$", font_size = 30).next_to(y_line_invis.get_midpoint(), RIGHT, buff = 0.1)
        
        x_line = DashedLine(
            ball.get_x()*RIGHT + ball.get_y()*UP,
            (ball.get_x()+1)*RIGHT + ball.get_y()*UP,
            stroke_width = 2
        )
        
        velocity = Arrow(
            ball.get_center(),
            ball.get_center() - UP*0.72148 + RIGHT,
            buff = 0,
            color = GREEN,
            tip_length = 0.2
        )
        velocity_label = Tex(r"$v$", font_size = 30, color = GREEN).next_to(velocity.get_end(), DR, buff = 0.1)

        alpha = Angle(
            velocity,
            y_line_extension,
            color = PURPLE,
            other_angle = True
        )
        alpha_label = Tex(r"$\alpha$", font_size = 30, color = PURPLE).next_to(alpha.get_midpoint(), DOWN, buff = 0.1)
    
        beta = Angle (
            velocity,
            x_line,
            color = ORANGE
        )
        beta_label = Tex(r"$\beta$", font_size = 30, color = ORANGE).next_to(beta, RIGHT, buff = 0.2)

        snells_law = MathTex("\\frac{\\sin(\\alpha)}{v}", "=", "\\text{constant}")
        snells_law.move_to(2*RIGHT + DOWN)
        
        conservation_isolates = ["m", "v", "g", "y", "2", "="]
        conservation = VGroup(
            MathTex("\\Delta\\,\\text{KE} = \\Delta \\text{PE}"),
            MathTex("\\frac{1}{2} mv^2 = mgy", substrings_to_isolate = conservation_isolates),
            MathTex("v = \\sqrt{2gy}", substrings_to_isolate = conservation_isolates)
        ).move_to(2*RIGHT + DOWN)
        energy_equation = conservation[1].copy()

        trig_isolates = ["\\alpha", "\\beta", "\\cos", "\\sin", "\\sec", "\\tan", "y", "=", "+"]
        trig = VGroup(
            MathTex(
                "\\sin \\alpha = \\cos \\beta",
                substrings_to_isolate = trig_isolates,
                tex_to_color_map = {"\\alpha" : PURPLE, "\\beta" : ORANGE}
            ),
            MathTex(
                "\\sin \\alpha = \\frac{1}{\\sec \\beta}",
                substrings_to_isolate = trig_isolates,
                tex_to_color_map = {"\\alpha" : PURPLE, "\\beta" : ORANGE}
            ),
            MathTex(
                "\\sin \\alpha = \\frac{1}{\\sqrt{1 + \\tan^2\\beta}}", 
                substrings_to_isolate = trig_isolates,
                tex_to_color_map = {"\\alpha" : PURPLE, "\\beta" : ORANGE}
            ),
            MathTex(
                "\\sin \\alpha = \\frac{1}{\\sqrt{1 + (y')^2}}",
                substrings_to_isolate = trig_isolates,
                tex_to_color_map = {"\\alpha" : PURPLE, "\\beta" : ORANGE}
            )
        ).move_to(2*RIGHT + DOWN)

        diff_isolates = ["\\text{constant}", "=", "\\frac{1}{\\sqrt{y[1 + (y')^2]}}", "y[1 + (y')^2]", "\\frac{1}{\\sqrt{(2gy)(1 + (y')^2)}}"]
        diff = VGroup(
            MathTex(
                "\\frac{\\sin\\alpha}{v} = \\frac{1}{\\sqrt{(2gy)(1 + (y')^2)}}",
                substrings_to_isolate=diff_isolates
            ),
            MathTex(
                "\\frac{1}{\\sqrt{(2gy)(1 + (y')^2)}} = \\text{constant}",
                substrings_to_isolate=diff_isolates,
                tex_to_color_map={"\\text{constant}": BLUE}
            ),
            MathTex(
                "\\frac{1}{\\sqrt{y[1 + (y')^2]}} = \\text{constant}",
                substrings_to_isolate=diff_isolates,
                tex_to_color_map = {"\\text{constant}": GREEN}
            ),
            MathTex(
                "\\sqrt{y[1 + (y')^2]} = \\text{constant}",
                substrings_to_isolate=diff_isolates,
                tex_to_color_map = {"\\text{constant}": YELLOW}
            ),
            MathTex(
                "y[1 + (y')^2] = \\text{constant}",
                substrings_to_isolate=diff_isolates,
                tex_to_color_map = {"\\text{constant}": RED}
            )
        ).move_to(2*RIGHT + DOWN)

        
        self.play(FadeIn(start_point))
        self.wait(0.5)
        self.play(FadeIn(end_point))
        self.wait(1)
        self.play(Create(cycloid))
        self.wait(4)
        self.play(
            Write(axes),
            FadeIn(ball),
            Create(y_line),
            Write(y_label)
        )
        self.wait(4)
        self.play(FadeOut(y_label), GrowArrow(velocity), Write(velocity_label))
        self.play(Create(y_line_extension))
        self.play(Create(alpha), Write(alpha_label))
        self.wait(3)
        self.play(Write(snells_law))
        self.wait(2)
        self.play(FadeOut(snells_law))
        self.wait(5)
        self.play(Write(conservation[0]))
        self.wait(1)
        self.play(
            FadeOut(conservation[0], shift = 0.5*UP), 
            FadeIn(energy_equation, shift = 0.5*UP)
        )
        self.wait(1)
        self.play(TransformMatchingTex(energy_equation, conservation[2]))
        self.wait(1)
        self.play(Circumscribe(energy_equation))
        self.wait(2)
        self.play(FadeOut(conservation[2]))
        self.wait(4)
        self.play(Create(x_line))
        self.play(Create(beta), Write(beta_label))
        self.wait(2)

        self.play(FadeIn(trig[0]))
        self.wait(1)
        for i in range(1, len(trig)-1 ):
            self.play(TransformMatchingTex(trig[i-1], trig[i]))
            self.wait(2)

        brace_x = BraceLabel(x_line, brace_direction = UP, text = "dx", font_size = 30, stroke_width = 3)
        brace_y = BraceLabel(y_line_extension, brace_direction = LEFT, text = "dy", font_size = 30, stroke_width = 3)
        self.play(FadeIn(brace_x))
        self.play(FadeIn(brace_y))
        self.wait(1)
        self.play(TransformMatchingTex(trig[-2], trig[-1]))
        self.play(FadeOut(brace_x, brace_y))
        self.wait(2)
        self.play(FadeOut(trig[-1]))
        self.wait(3)

        self.play(FadeIn(diff[0]))
        self.wait(2)
        for i in range(1, len(diff) ):
            self.play(TransformMatchingTex(diff[i-1], diff[i]))
            self.wait(3)
        self.play(FocusOn(diff[-1]))
        self.play(
            FadeOut(
                start_point, 
                end_point,
                axes,
                velocity, velocity_label,
                y_line_extension, 
                y_line, 
                x_line,
                alpha, alpha_label,
                beta, beta_label, 
                ball,
                cycloid
            )
        )
        self.wait(2)
        self.play(diff[-1].animate.move_to(UP))
        
        self.wait(1)
        

Manim Community v0.19.0

[06/09/25 21:45:08] INFO     Animation 0 : Using cached data (hash :                           ]8;id=469618;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=99091;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             360981556_2640751701_223132457)                                                       

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=228727;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=171401;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3929419539_422628456_4243807291)                                                      

                    INFO     Animation 2 : Using cached data (hash :                           ]8;id=513319;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=575331;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3929419539_3470186326_3758448455)                                                     

                    INFO     Animation 3 : Using cached data (hash :                           ]8;id=370709;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=446981;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3929419539_1121617232_385121484)                                                      

...

[06/09/25 21:46:11] INFO     Animation 57 : Using cached data (hash :                          ]8;id=595627;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=674195;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3929419539_1121617232_592005393)                                                      

                    INFO     Combining to Movie file.                                      ]8;id=316697;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=974602;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=533930;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=566149;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\ProblemSolving.mp4'                                
                                                                                                                   

                    INFO     The partial movie directory is full (> 100 files). Therefore, ]8;id=77308;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=407792;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#856\856]8;;\
                             manim has removed the 20 oldest file(s). You can change this                          
                             behaviour by changing max_files_cached in config.                                     

                    INFO     Rendered ProblemSolving                                                   ]8;id=598742;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene.py\scene.py]8;;\:]8;id=941490;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene.py#255\255]8;;\
                             Played 58 animations                                                                  

In [58]:
%%manim -qh Solution

class Solution(Scene):
    def construct(self):
        diff_isolates = [" c ", "=", " y ", " 1 ", " + ", "\\frac{dy}{dx}", "\\frac{y}{c-y}"]  
        diff = VGroup(
            MathTex(
                " y [ 1  +  (y')^2] = \\text{constant}",
                substrings_to_isolate = diff_isolates,
                tex_to_color_map = {"\\text{constant}": RED}
            ),
            MathTex(
                " y \\left[ 1  +  \\left(\\frac{dy}{dx}\\right)^2\\right] = c ",
                substrings_to_isolate = diff_isolates,
                tex_to_color_map = {" c ": RED}
            ),
            MathTex(
                " \\left(\\frac{y}{ c -y}\\right)^{1/2} dy = dx",
                substrings_to_isolate = diff_isolates,
                tex_to_color_map = {" c ": RED}
            ),
            MathTex(
                " \\int\\left(\\frac{y}{ c -y}\\right)^{1/2} dy = \\int dx",
                substrings_to_isolate = diff_isolates,
                tex_to_color_map = {" c ": RED}
            )
        ).move_to(UP)
        final_eq = diff[-2].copy()

        param_isolates = [" c ", " \\phi ", "\\sin{\\phi}", "\\sin^2{\\phi}"]
        param = VGroup(
            MathTex(
                "\\left(\\frac{y}{ c -y}\\right)^{1/2}", "dy = dx",
                substrings_to_isolate = param_isolates,
                tex_to_color_map = {" c ": RED}
            )
        ).move_to(UP)

        param_brace = Brace(param[0][:-1])
        param_brace_label = MathTex(
            " \\tan \\phi ",
            substrings_to_isolate = [" \\phi "],
            tex_to_color_map = {" \\phi " : GREEN},
            font_size = 40
        ).next_to(param_brace, DOWN, buff = 0.2)

        y_eq_isolates = [" c ", " \\phi ", " \\sin ", " \\cos ", "\\frac{c}{2}", "=", " y "]
        y_eq = VGroup(
            MathTex(
                "d y = 2  c  \\sin  \\phi  \\cos  \\phi  \\, d  \\phi  ",
                substrings_to_isolate = y_eq_isolates,
                tex_to_color_map = {" c ": RED, " \\phi " : GREEN}
            ),
            MathTex(
                "y =  c  \\sin^2(  \\phi  ) +  C",
                substrings_to_isolate = y_eq_isolates,
                tex_to_color_map = {" c ": RED, " \\phi " : GREEN}
            ),
            MathTex(
                "y =  \\frac{c}{2}  (1 - \\cos  2  \\phi ) ", "+ C",
                substrings_to_isolate = y_eq_isolates,
                color = BLUE
            )
        ).move_to(DL + LEFT)

        x_eq_isolates = [" x ", "=", " \\tan ", " \\sin ", " \\phi ", " \\cos ", "\\frac{c}{2}"]
        x_eq = VGroup(
            MathTex(
                "d x = \\tan  \\phi  dy",
                substrings_to_isolate = x_eq_isolates,
                tex_to_color_map = {" c " : RED, " \\phi " : GREEN}
            ),
            MathTex(
                "d x = \\tan  \\phi  \\times 2  c  \\sin  \\phi  \\cos  \\phi  \\, d   \\phi ",
                substrings_to_isolate = x_eq_isolates,
                tex_to_color_map = {" c " : RED, " \\phi " : GREEN}
            ),
            MathTex(
                "d x = 2  c  \\sin  ^2(   \\phi  )\\,  d  \\phi  ",
                substrings_to_isolate = x_eq_isolates,
                tex_to_color_map = {" c " : RED, " \\phi " : GREEN}
            ),
            MathTex(
                "d x = c  (1  -  \\cos  2  \\phi  )  \\, d  \\phi ",
                substrings_to_isolate = x_eq_isolates,
                tex_to_color_map = {" c " : RED, " \\phi " : GREEN}
            ),
            MathTex(
                " x = \\frac{c}{2}  (2  \\phi  -  \\sin  2  \\phi  )  ", "+ C",
                substrings_to_isolate = x_eq_isolates,
                color = ORANGE
            )
        ).next_to(y_eq[-1], DOWN)

        solution = VGroup(y_eq[-1], x_eq[-1])

        boundary_brace = Brace(solution, RIGHT)
        boundary_condition = MathTex(
            "x = y = 0 \\text{ at } \\phi = 0",
            substrings_to_isolate = ["\\phi"],
            tex_to_color_map = {"\\phi":GREEN},
            font_size = 40
        ).next_to(boundary_brace, RIGHT, buff = 0.3)


        sep_sol = MathTex(
                " -\\sqrt{  c  -y} \\sqrt{y} -  c  \\tan^{-1}\\left(\\frac{\\sqrt{  c  -y}}{\\sqrt{y}}\\right) = x +  C  ",
                color = BLUE, font_size = 40
            ).next_to(diff[-1], DOWN)
        
        self.add(diff[0])
        self.wait(2)
        for i in range(1, len(diff) ):
            self.play(TransformMatchingTex(diff[i-1], diff[i]))
            self.wait(2)
        self.play(FadeIn(sep_sol))
        self.wait(3)
        self.play(FadeOut(sep_sol))
        self.wait(2)
        self.play(TransformMatchingTex(diff[-1], param[0]))
        self.remove(diff[-1])
        self.add(param[0])
        self.wait()
        self.play(FadeIn(param_brace))
        self.wait(0.5)
        self.play(FadeIn(param_brace_label))
        self.wait(1)
        self.play(FadeIn(y_eq[0]), FadeOut(param_brace, param_brace_label))
        self.wait(4)
        for i in range(1, len(y_eq)):
            self.play(TransformMatchingTex(y_eq[i-1], y_eq[i]))
            self.wait(3)

        self.wait(1)
        self.play(FadeIn(x_eq[0]))
        self.wait(4)
        for i in range(1, len(x_eq)):
            self.play(TransformMatchingTex(x_eq[i-1], x_eq[i]))
            self.wait(3)
        self.wait(2)
        self.play(FadeIn(boundary_brace, boundary_condition))
        self.wait(3)
        self.play(FadeOut(x_eq[-1][-1], y_eq[-1][-1], boundary_brace, boundary_condition))
        x_eq[-1].remove(x_eq[-1][-1])
        y_eq[-1].remove(y_eq[-1][-1])
        self.wait(3)
        
        final_y = MathTex(
            "  y  =  a  (  1  -  \\cos  \\theta  )",
            substrings_to_isolate = y_eq_isolates,
            color = BLUE
        ).move_to(y_eq[-1])
        final_x = MathTex(
            "  x  =  a  (  \\theta  -  \\sin  \\theta  )",
            substrings_to_isolate = x_eq_isolates,
            color = ORANGE
        ).move_to(x_eq[-1])
        
        self.play(TransformMatchingTex(x_eq[-1], final_x))
        self.play(TransformMatchingTex(y_eq[-1], final_y))
        self.wait(4)
        self.play(FadeOut(param[0]))
        self.play(VGroup(final_x, final_y).animate.arrange(RIGHT, buff = 1).to_edge(UP, buff = 2))
        self.play(FocusOn(VGroup(final_x, final_y)))
        self.wait(3)
        self.play(FadeOut(final_x, shift = 0.5*UP), FadeOut(final_y, shift = 0.5*UP))
        self.wait(5)

        print("X = (", final_x.get_x(), final_x.get_y(), ")")
        print("Y = (", final_y.get_x(), final_y.get_y(), ")")
    



Manim Community v0.19.0

[06/10/25 00:52:57] INFO     Writing d  x  =  c   (1  -   \cos   2   \phi   )  \, d   \phi  ]8;id=912688;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\utils\tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=579819;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\utils\tex_file_writing.py#111\111]8;;\
                             to media\Tex\34c0261410af74b2.tex                                                     

[06/10/25 00:52:58] INFO     Writing (1  - to media\Tex\a5642349502ff9dc.tex                ]8;id=374531;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\utils\tex_file_writing.py\tex_file_writing.py]8;;\:]8;id=497059;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\utils\tex_file_writing.py#111\111]8;;\

[06/10/25 00:53:00] INFO     Animation 0 : Using cached data (hash :                           ]8;id=144487;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=413732;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3529212410_2408024828_2861163771)                                                     

                    INFO     Animation 1 : Using cached data (hash :                           ]8;id=824261;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py\cairo_renderer.py]8;;\:]8;id=808253;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\renderer\cairo_renderer.py#89\89]8;;\
                             3040924799_306958822_2873729818)                                                      

...

X = ( -2.1841729999999995 1.750934 )
Y = ( 2.1671068500000006 1.750934 )


                    INFO     Combining to Movie file.                                      ]8;id=821649;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=911530;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#739\739]8;;\

[06/10/25 00:53:28] INFO                                                                   ]8;id=650545;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=688625;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             'C:\Users\amine\Documents\ManimCE\Projects\Brachistochrone\me                         
                             dia\videos\Brachistochrone\1080p60\Solution.mp4'                                      
                                                                                                                   

                    INFO     The partial movie directory is full (> 100 files). Therefore, ]8;id=571665;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=270848;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene_file_writer.py#856\856]8;;\
                             manim has removed the 7 oldest file(s). You can change this                           
                             behaviour by changing max_files_cached in config.                                     

                    INFO     Rendered Solution                                                         ]8;id=165760;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene.py\scene.py]8;;\:]8;id=98881;file://C:\Users\amine\Documents\ManimCE\mce\Lib\site-packages\manim\scene\scene.py#255\255]8;;\
                             Played 48 animations                                                                  